# Co-creating music with SymphonyNet

## Install the requirements

In [ ]:
!git clone https://github.com/symphonynet/SymphonyNet.git

Cloning into 'SymphonyNet'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 93 (delta 3), reused 9 (delta 0), pack-reused 78
Unpacking objects: 100% (93/93), done.


In [ ]:
!cat SymphonyNet/requirements.txt | xargs -n 1 -L 1 pip install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 776.8 MB 4.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.0+cu116 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.13.0+cu116 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 18.3 MB/s 
     |████████████████████████████████| 118 kB 76.0 MB/s 
 

## Model preparation

In [ ]:
MAX_POS_LEN = 4096
PI_LEVEL = 2
IGNORE_META_LOSS = 1
RATIO = 4
BPE = "_bpe" # or ""

DATA_BIN=f"linear_{MAX_POS_LEN}_chord{BPE}_hardloss{IGNORE_META_LOSS}"
CHECKPOINT_SUFFIX=f"{DATA_BIN}_PI{PI_LEVEL}"
DATA_BIN_DIR=f"SymphonyNet/data/model_spec/{DATA_BIN}/bin/"
DATA_VOC_DIR=f"SymphonyNet/data/model_spec/{DATA_BIN}/vocabs/"
from SymphonyNet.src.fairseq.gen_utils import process_prime_midi, gen_one, get_trk_ins_map, get_note_seq, note_seq_to_midi_file, music_dict


In [ ]:
%cd ../

/content


In [ ]:
music_dict.load_vocabs_bpe(DATA_VOC_DIR, 'SymphonyNet/data/bpe_res/' if BPE == '_bpe' else None)


Initialize the model and load pretrained parameters. (You should first save the provided [ckpt file](https://drive.google.com/file/d/1xpkj_qN4MdLRkBdCXmfGjuWWjnTN1Og0/view) into your google drive.)

In [ ]:
!pwd

/content


In [ ]:
model_name = 'jazz_bpe_model.pt'
from fairseq.models import FairseqLanguageModel
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
custom_lm = FairseqLanguageModel.from_pretrained('.', 
    checkpoint_file=f'drive/MyDrive/{model_name}', 
    data_name_or_path=DATA_BIN_DIR, 
    user_dir="SymphonyNet/src/fairseq/linear_transformer_inference")

Mounted at /content/drive


In [ ]:
orig_lm = FairseqLanguageModel.from_pretrained('.', 
    checkpoint_file=f'drive/MyDrive/orig_model.pt', 
    data_name_or_path=DATA_BIN_DIR, 
    user_dir="SymphonyNet/src/fairseq/linear_transformer_inference")

In [ ]:
for embedding_layers in ['wEvte', 'wTrke','wDure','wRpe','wMpe']:

  a = getattr(orig_lm.models[0].decoder, embedding_layers)
  setattr(custom_lm.models[0].decoder, embedding_layers, a) 


In [ ]:
m = custom_lm.models[0]
m.cuda()
m.eval()

## Prepare prime MIDI

In [ ]:
midi_name = 'SymphonyNet/test.mid'
max_measure_cnt = 5
max_chord_measure_cnt = 0
prime, ins_label = process_prime_midi(midi_name, max_measure_cnt, max_chord_measure_cnt)

## Generation

In [ ]:
%cd SymphonyNet

!python3 src/fairseq/gen_batch.py test.mid 5 0 1

/content/SymphonyNet
Traceback (most recent call last):
  File "src/fairseq/gen_batch.py", line 18, in <module>
    custom_lm = FairseqLanguageModel.from_pretrained('.', 
  File "/usr/local/lib/python3.8/dist-packages/fairseq/models/fairseq_model.py", line 261, in from_pretrained
    x = hub_utils.from_pretrained(
  File "/usr/local/lib/python3.8/dist-packages/fairseq/hub_utils.py", line 70, in from_pretrained
    models, args, task = checkpoint_utils.load_model_ensemble_and_task(
  File "/usr/local/lib/python3.8/dist-packages/fairseq/checkpoint_utils.py", line 278, in load_model_ensemble_and_task
    raise IOError("Model file not found: {}".format(filename))
OSError: Model file not found: ./ckpt/checkpoint_last_linear_4096_chord_bpe_hardloss1_PI2.pt


In [ ]:
import time
while(True):
  try: 
    while(True):
      try:
        generated, ins_logits = gen_one(m, prime, MIN_LEN = 1024)
        break
      except Exception as e:
        print(e)
        continue
    trk_ins_map = get_trk_ins_map(generated, ins_logits)
    note_seq = get_note_seq(generated, trk_ins_map)
    timestamp = time.strftime("%m-%d_%H-%M-%S", time.localtime()) 
    output_name = f'output_prime{max_measure_cnt}_chord{max_chord_measure_cnt}_{timestamp}.mid'
    note_seq_to_midi_file(note_seq, output_name)
    break;
  except Exception as e:
    print(e)
    continue


  4%|▍         | 169/3895 [00:02<00:47, 78.27it/s]


Invalid generation: Generated excerpt too short.


  4%|▎         | 141/3895 [00:01<00:49, 76.40it/s]


Invalid generation: Generated excerpt too short.


 39%|███▊      | 1507/3895 [00:19<00:30, 77.74it/s]


('Invalid generation: unknown token: ', ('<unk>', 1, 1))


 21%|██▏       | 831/3895 [00:10<00:39, 77.88it/s]


('Invalid generation: unknown token: ', ('<bos>', 1, 1))


 26%|██▌       | 1018/3895 [00:13<00:37, 77.60it/s]


In [ ]:
#  else:
       #     assert False, ('Invalid generation: unknown token: ', (ee, d, t))

## Audio Display

In [ ]:
!sudo apt install -y fluidsynth

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 20 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfluidsynth1 amd64 1.1.9-1 [137 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:4 http://archi

In [ ]:
!pip install --upgrade pyfluidsynth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pretty_midi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 28.9 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591954 sha256=8cd9963be14f7dd51a902dcfbfd1fd96c331d25751ae91aa0e511366962a022c
  Stored in directory: /root/.cache/pip/wheels/2a/5a/e3/30eeb9a99350f3f7e21258fcb132743eef1a4f49b3505e76b6
Successfully built pretty-midi


In [ ]:
import fluidsynth
import pretty_midi
from IPython import display
_SAMPLING_RATE = 16000
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=300):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [ ]:
pm = pretty_midi.PrettyMIDI(output_name)
display_audio(pm)